#🎯 Objective
Rotate a log file when it gets large (simulate size-based rotation).

Archive old log files (e.g., compress them).

Clean up logs older than X days (optional).

#✅ Step-by-Step Guide in Google Colab

#🔹 Step 1: Simulate a Sample Log File

In [1]:
# Create a dummy log directory and a sample log file
!mkdir -p logs
!echo "Log entry 1" > logs/app.log
!for i in $(seq 2 100); do echo "Log entry $i" >> logs/app.log; done

# Check size and content
!ls -lh logs
!tail logs/app.log


total 4.0K
-rw-r--r-- 1 root root 1.3K Jun  9 07:50 app.log
Log entry 91
Log entry 92
Log entry 93
Log entry 94
Log entry 95
Log entry 96
Log entry 97
Log entry 98
Log entry 99
Log entry 100


#📝 Explanation:

mkdir -p logs: Makes a log directory.

echo and for i in $(seq ...): Appends log entries.

ls -lh: Lists file size.

tail: Shows the last few lines.

#🔹 Step 2: Write the Log Rotation Shell Script

In [2]:
%%writefile rotate_logs.sh
#!/bin/bash

# Set variables
LOG_DIR="logs"
LOG_FILE="$LOG_DIR/app.log"
ARCHIVE_DIR="$LOG_DIR/archive"
DATE=$(date +"%Y-%m-%d_%H-%M-%S")

# Create archive directory if not exists
mkdir -p "$ARCHIVE_DIR"

# Set max log size in bytes (e.g., 1 KB = 1024 bytes for testing)
MAX_SIZE=1024

# Get actual file size
FILE_SIZE=$(stat -c%s "$LOG_FILE")

# Rotate if size exceeds threshold
if [ "$FILE_SIZE" -gt "$MAX_SIZE" ]; then
    echo "Rotating log: $LOG_FILE is $FILE_SIZE bytes (> $MAX_SIZE bytes)"

    # Move log to archive with timestamp
    mv "$LOG_FILE" "$ARCHIVE_DIR/app_$DATE.log"

    # Create a new empty log file
    touch "$LOG_FILE"
else
    echo "No rotation needed. Log file size: $FILE_SIZE bytes"
fi


Writing rotate_logs.sh


| Line                      | What It Does                           |
| ------------------------- | -------------------------------------- |
| `LOG_FILE`, `ARCHIVE_DIR` | Paths to manage logs                   |
| `mkdir -p`                | Ensures archive folder exists          |
| `stat -c%s`               | Gets file size in bytes                |
| `mv`                      | Moves (renames) log file for archiving |
| `touch`                   | Creates a fresh, empty log             |


#🔹 Step 3: Run the Log Rotation Script

In [3]:
# Give execute permission and run
!chmod +x rotate_logs.sh
!./rotate_logs.sh

# Check files
!ls -lh logs
!ls -lh logs/archive


Rotating log: logs/app.log is 1292 bytes (> 1024 bytes)
total 4.0K
-rw-r--r-- 1 root root    0 Jun  9 07:53 app.log
drwxr-xr-x 2 root root 4.0K Jun  9 07:53 archive
total 4.0K
-rw-r--r-- 1 root root 1.3K Jun  9 07:50 app_2025-06-09_07-53-15.log


#🔹 Step 4: Test Again by Increasing Log Size



In [4]:
# Add more logs to exceed the 1KB limit
!for i in $(seq 1 200); do echo "New log entry $i" >> logs/app.log; done

# Run the rotation again
!./rotate_logs.sh

# Check archive
!ls -lh logs/archive


Rotating log: logs/app.log is 3492 bytes (> 1024 bytes)
total 8.0K
-rw-r--r-- 1 root root 1.3K Jun  9 07:50 app_2025-06-09_07-53-15.log
-rw-r--r-- 1 root root 3.5K Jun  9 07:53 app_2025-06-09_07-53-52.log
